In [ ]:
%load_ext autoreload
%autoreload 2

import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import processor, hist

from processor.FCNC_cutflow import dielectron_mass
from Tools.config_helpers import loadConfig
from klepto.archives import dir_archive

lumi = {2016: 36, 2017: 42, 2018: 60}

In [21]:
from processor.default_accumulators import desired_output, add_processes_to_output

from Tools.helpers import get_samples, cutflow_scale_and_merge
from Tools.config_helpers import redirector_ucsd, redirector_fnal
from Tools.nano_mapping import make_fileset, nano_mapping

from processor.meta_processor import get_sample_meta

overwrite = True
local = True

# load the config and the cache
cfg = loadConfig()

cacheName = 'dielectron_mass'
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cacheName), serialized=True)

year = 2018

# get a python dictionary of all NanoAOD samples
# The samples definitions can be found in data/samples.yaml
samples = get_samples(year)

# make a fileset, taking the definitions in Tools/nano_mapping.py
nano_mappings = nano_mapping(year)
fileset = make_fileset(['hct', 'hut', 'TT'], year, redirector=redirector_ucsd, small=False)
meta = get_sample_meta(fileset, samples)
# 'DY', 'Rares', 'TT/TW', 'TTH', 'TTW', 'TTZ', 'TTVV', 'W', 'X+g'
# in order for cutflows to work we need to add every process to the output accumulator
add_processes_to_output(fileset, desired_output)

histograms = sorted(list(desired_output.keys()))

if local:

    exe_args = {
        'workers': 16,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.futures_executor

else:
    from Tools.helpers import get_scheduler_address
    from dask.distributed import Client, progress

    scheduler_address = get_scheduler_address()
    c = Client(scheduler_address)

    exe_args = {
        'client': c,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.dask_executor


if not overwrite:
    cache.load()

if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
    output = cache.get('simple_output')
    
else:
    print ("I'm running now")

    output = processor.run_uproot_job(
        fileset,
        "Events",
        dielectron_mass(year=year, variations=[], accumulator=desired_output),
        exe,
        exe_args,
        chunksize=250000,
    )

    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()

In [ ]:
print(output['totalEvents']['all']/1e6)

In [ ]:
fileset = make_fileset(['hct', 'hut', 'TT', 'TW', 'W'], year, redirector=redirector_ucsd, small=False)
meta = get_sample_meta(fileset, samples)

In [ ]:
if year == 2016:
    output['hct'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hct-MadGraph5-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM'] + output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hct-MadGraph5-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM']
    output['hut'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hut-MadGraph5-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM'] + output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hut-MadGraph5-pythia8/RunIISummer16NanoAODv7-PUMoriond17_Nano02Apr2020_102X_mcRun2_asymptotic_v8-v1/NANOAODSIM']

if year == 2017:
    output['hct'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hct_TuneCP5-MadGraph5-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_tauDecays_102X_mc2017_realistic_v8-v1/NANOAODSIM'] + output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hct_TuneCP5-MadGraph5-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_tauDecays_102X_mc2017_realistic_v8-v1/NANOAODSIM']
    output['hut'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hut_TuneCP5-MadGraph5-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_tauDecays_102X_mc2017_realistic_v8-v1/NANOAODSIM'] + output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hut_TuneCP5-MadGraph5-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_tauDecays_102X_mc2017_realistic_v8-v1/NANOAODSIM'] 

if year == 2018:
    output['hct'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hct_TuneCP5-MadGraph5-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_tauDecays_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hct_TuneCP5-MadGraph5-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_tauDecays_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
    output['hut'] = output['/TT_FCNC-TtoHJ_aTleptonic_HToWWZZtautau_eta_hut_TuneCP5-MadGraph5-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_tauDecays_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']+ output['/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hut_TuneCP5-MadGraph5-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_tauDecays_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
    #output['TT'] = output['/TTJets_SingleLeptFromT_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + ['/TTJets_SingleLeptFromTbar_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
    #output['DY'] = output['/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + output['/DYJetsToLL_M-10to50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM']
    #output['Rares'] = output['/GluGluHToZZTo4L_M125_13TeV_powheg2_JHUGenV7011_pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + output['/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/VHToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + output['/WWW_4F_TuneCP5_13TeV-amcatnlo-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/WWZ_TuneCP5_13TeV-amcatnlo-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/WZTo3LNu_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + output['/WZG_TuneCP5_13TeV-amcatnlo-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + output['/WWG_TuneCP5_13TeV-amcatnlo-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/WZZ_TuneCP5_13TeV-amcatnlo-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/ZZTo4L_TuneCP5_13TeV_powheg_pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext2-v1/NANOAODSIM'] + output['/ZZZ_TuneCP5_13TeV-amcatnlo-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/TTTJ_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext2-v1/NANOAODSIM'] + output['/TTTW_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM']
    #output['TT/TW/W'] = output['/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext2-v1/NANOAODSIM'] + output['/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_EXT1_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_EXT1_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/ST_tWll_5f_LO_TuneCP5_PSweights_13TeV-madgraph-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + output['/WWTo2L2Nu_DoubleScattering_13TeV-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
    #output['TTH'] = output['/ttHToNonbb_M125_TuneCP5_13TeV-powheg-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
    #output['TTW'] = output['/TTWJetsToLNu_TuneCP5_13TeV-amcatnloFXFX-madspin-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM']
    #output['TTZ'] = output['/TTZToLLNuNu_M-10_TuneCP5_13TeV-amcatnlo-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/TTZToLL_M-1to10_TuneCP5_13TeV-amcatnlo-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM']
    #output['TTVV'] = output['/TTHH_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/TTWW_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext2-v1/NANOAODSIM'] + output['/TTWZ_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/TTZH_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/TTZZ_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM']
    #output['X+g'] = output['/TTGamma_SingleLept_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + output['/TTGamma_Dilept_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21-v1/NANOAODSIM'] + output['/WGToLNuG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM'] + output['/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext2-v1/NANOAODSIM'] + output['/TGJets_TuneCP5_13TeV_amcatnlo_madspin_pythia8/RunIIAutumn18NanoAODv7-Nano02Apr2020_102X_upgrade2018_realistic_v21_ext1-v1/NANOAODSIM']

In [ ]:
from Tools.helpers import getCutFlowTable, cutflow_scale_and_merge


lines = ['entry', '"skim"', 'lepton kinematics', 'lepton selection', 'triggers', 'filter', 'ss', 'SS onZ veto', 'two jets', 'MET > 50']

#output2 = cutflow_scale_and_merge(output, meta, fileset, nano_mappings, lumi=lumi[year])
df = getCutFlowTable(output, processes=['hct', 'hut'], lines=lines, significantFigures=7, signal=['hct', 'hut'])
df

In [ ]:
from Tools.helpers import getCutFlowTable, cutflow_scale_and_merge


lines = ['entry', '"skim"', '>1 tight', 'triggers', 'filter', 'ss', 'mass vetoes', 'two jets', 'MET > 50']

#output2 = cutflow_scale_and_merge(output, meta, fileset, nano_mappings, lumi=lumi[year])
df = getCutFlowTable(output, processes=['hct', 'hut', 'DY', 'TT/TW/W', 'TTH', 'TTW', 'TTZ', 'TTVV', 'X+g', 'Rares'], lines=lines, significantFigures=7, signal=['hct', 'hut'])
df
#'DY', 'TT/TW', 'WZ/ZZ',

In [ ]:
import gzip
import pickle
import cloudpickle
outname = 'more_backgrounds_and_signal_'+str(year)
os.system("mkdir -p histos/")
print('Saving output in %s...'%("histos/" + outname + ".pkl.gz"))
with gzip.open("histos/" + outname + ".pkl.gz", "wb") as fout:
    cloudpickle.dump(output, fout)
print('Done!')

In [ ]:
# import the plotting libararies: matplotlib and mplhep

import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

import numpy as np


# load the functions to make a nice plot from the output histograms
# and the scale_and_merge function that scales the individual histograms
# to match the physical cross section

from plots.helpers import scale_and_merge

# define a few axes that we can use to rebin our output histograms

N_bins         = hist.Bin('multiplicity', r'$N$', 10, -0.5, 9.5)
N_bins_red     = hist.Bin('multiplicity', r'$N$', 5, -0.5, 4.5)
pt_bins        = hist.Bin('pt', r'$p_{T}\ (GeV)$', np.array([15, 40, 60, 80, 100, 200, 300]))
pt_fine_bins   = hist.Bin('pt', r'$p_{T}\ (GeV)$', 300, 0, 300)
pt_rebin       = hist.Bin('pt', r'$p_{T}\ (GeV)$', 280, 20, 300)
pt_rebin2      = hist.Bin('pt', r'$p_{T}\ (GeV)$', 60, 0, 300)
eta_bins       = hist.Bin('eta', r'$\eta $', np.array([0, 0.8, 1.479, 2.5]))
eta_rebin      = hist.Bin('eta', r'$\eta $', 25, -2.5, 2.5)    
phi_bins       = hist.Bin('phi', r'$\phi $', 16, -3.2, 3.2)
mass_bins      = hist.Bin('mass', r'$mass (GeV/c^2)$', 60, 0, 300)

# 1D Histograms

In [ ]:
from yahist import Hist1D, Hist2D

In [ ]:
def get_total(histos, keys):
        tmp = Hist1D.from_bincounts(np.zeros(len(histos[keys[0]].counts)), histos[keys[0]].edges, )
        for key in keys:
            tmp += histos[key]
        return tmp

def add_uncertainty(hist, ax, ratio=False):
    opts = {'step': 'post', 'label': 'Uncertainty', 'hatch': '///',
                    'facecolor': 'none', 'edgecolor': (0, 0, 0, .5), 'linewidth': 0, 'zorder':10.}
    
    if ratio:
        down = np.ones(len(hist.counts)) - hist.errors/hist.counts
        up = np.ones(len(hist.counts)) + hist.errors/hist.counts
    else:
        down = hist.counts-hist.errors
        up = hist.counts+hist.errors
    ax.fill_between(x=hist.edges, y1=np.r_[down, down[-1]], y2=np.r_[up, up[-1]], **opts)

In [ ]:
tmp1 = scale_and_merge(output['electron_data1'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp1 = tmp1.rebin('pt', pt_rebin2)
tmp1 = tmp1.rebin('eta', eta_rebin)

h1_hct = Hist1D.from_bincounts(
    tmp1.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp1.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)


h1_hut = Hist1D.from_bincounts(
    tmp1.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp1.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h1_hct=h1_hct*0.01
h1_hut=h1_hut*0.01

h1_tt = Hist1D.from_bincounts(
    tmp1.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp1.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp2 = scale_and_merge(output['electron_data2'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp2 = tmp2.rebin('pt', pt_rebin2)
tmp2 = tmp2.rebin('eta', eta_rebin)

h2_hct = Hist1D.from_bincounts(
    tmp2.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp2.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp2.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h2_hut = Hist1D.from_bincounts(
    tmp2.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp2.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp2.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h2_hct=h2_hct*0.01
h2_hut=h2_hut*0.01

h2_tt = Hist1D.from_bincounts(
    tmp2.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp2.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp2.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp3 = scale_and_merge(output['electron_data3'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp3 = tmp3.rebin('pt', pt_rebin2)
tmp3 = tmp3.rebin('eta', eta_rebin)

h3_hct = Hist1D.from_bincounts(
    tmp3.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp3.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp3.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h3_hut = Hist1D.from_bincounts(
    tmp3.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp3.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp3.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h3_hct=h3_hct*0.01
h3_hut=h3_hut*0.01

h3_tt = Hist1D.from_bincounts(
    tmp3.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp3.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp3.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp4 = scale_and_merge(output['electron_data4'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp4 = tmp4.rebin('pt', pt_rebin2)
tmp4 = tmp4.rebin('eta', eta_rebin)

h4_hct = Hist1D.from_bincounts(
    tmp4.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp4.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp4.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h4_hut = Hist1D.from_bincounts(
    tmp4.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp4.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp4.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h4_hct=h4_hct*0.01
h4_hut=h4_hut*0.01

h4_tt = Hist1D.from_bincounts(
    tmp4.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp4.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp4.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp5 = scale_and_merge(output['electron_data5'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp5 = tmp5.rebin('pt', pt_rebin)
tmp5 = tmp5.rebin('eta', eta_rebin)

h5_hct = Hist1D.from_bincounts(
    tmp5.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp5.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp5.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h5_hut = Hist1D.from_bincounts(
    tmp5.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp5.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp5.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h5_hct=h5_hct*0.01
h5_hut=h5_hut*0.01

h5_tt = Hist1D.from_bincounts(
    tmp5.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp5.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp5.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp6 = scale_and_merge(output['electron_data6'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp6 = tmp6.rebin('pt', pt_rebin)
tmp6 = tmp6.rebin('eta', eta_rebin)

h6_hct = Hist1D.from_bincounts(
    tmp6.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp6.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp6.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h6_hut = Hist1D.from_bincounts(
    tmp6.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp6.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp6.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h6_hct=h6_hct*0.01
h6_hut=h6_hut*0.01

h6_tt = Hist1D.from_bincounts(
    tmp6.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp6.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp6.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp7 = scale_and_merge(output['electron_data7'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp7 = tmp7.rebin('pt', pt_rebin)
tmp7 = tmp7.rebin('eta', eta_rebin)

h7_hct = Hist1D.from_bincounts(
    tmp7.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp7.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp7.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h7_hut = Hist1D.from_bincounts(
    tmp7.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp7.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp7.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h7_hct=h7_hct*0.01
h7_hut=h7_hut*0.01

h7_tt = Hist1D.from_bincounts(
    tmp7.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp7.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp7.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp8 = scale_and_merge(output['electron_data8'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp8 = tmp8.rebin('pt', pt_rebin)
tmp8 = tmp8.rebin('eta', eta_rebin)

h8_hct = Hist1D.from_bincounts(
    tmp8.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp8.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp8.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h8_hut = Hist1D.from_bincounts(
    tmp8.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp8.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp8.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h8_hct=h8_hct*0.01
h8_hut=h8_hut*0.01

h8_tt = Hist1D.from_bincounts(
    tmp8.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp8.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp8.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp9 = scale_and_merge(output['electron_data9'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp9 = tmp9.rebin('pt', pt_rebin2)
tmp9 = tmp9.rebin('eta', eta_rebin)

h9_hct = Hist1D.from_bincounts(
    tmp9.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp9.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp9.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h9_hut = Hist1D.from_bincounts(
    tmp9.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp9.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp9.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h9_hct=h9_hct*0.01
h9_hut=h9_hut*0.01

h9_tt = Hist1D.from_bincounts(
    tmp9.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp9.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp9.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp10 = scale_and_merge(output['electron_data10'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp10 = tmp10.rebin('pt', pt_rebin2)
tmp10 = tmp10.rebin('eta', eta_rebin)

h10_hct = Hist1D.from_bincounts(
    tmp10.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp10.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp10.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h10_hut = Hist1D.from_bincounts(
    tmp10.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp10.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp10.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h10_hct=h10_hct*0.01
h10_hut=h10_hut*0.01

h10_tt = Hist1D.from_bincounts(
    tmp10.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp10.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp10.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp11 = scale_and_merge(output['electron_data11'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp11 = tmp11.rebin('pt', pt_rebin2)
tmp11 = tmp11.rebin('eta', eta_rebin)

h11_hct = Hist1D.from_bincounts(
    tmp11.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp11.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp11.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h11_hut = Hist1D.from_bincounts(
    tmp11.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp11.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp11.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h11_hct=h11_hct*0.01
h11_hut=h11_hut*0.01

h11_tt = Hist1D.from_bincounts(
    tmp11.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp11.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp11.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp12 = scale_and_merge(output['electron_data12'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp12 = tmp12.rebin('pt', pt_rebin2)
tmp12 = tmp12.rebin('eta', eta_rebin)

h12_hct = Hist1D.from_bincounts(
    tmp12.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp12.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp12.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h12_hut = Hist1D.from_bincounts(
    tmp12.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp12.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp12.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h12_hct=h12_hct*0.01
h12_hut=h12_hut*0.01

h12_tt = Hist1D.from_bincounts(
    tmp12.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp12.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp12.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

In [ ]:
hists = {'lead_ele_hct': h1_hct,
         'lead_ele_hut': h1_hut,
         'lead_ele_tt': h1_tt,
         'lead_ele2_hct': h2_hct,
         'lead_ele2_hut': h2_hut,
         'lead_ele2_tt': h2_tt,
         'lead_muon_hct': h3_hct,
         'lead_muon_hut': h3_hut,
         'lead_muon_tt': h3_tt,
         'lead_muon2_hct': h4_hct,
         'lead_muon2_hut': h4_hut,
         'lead_muon2_tt': h4_tt,
         'sublead_ele_hct': h5_hct,
         'sublead_ele_hut': h5_hut,
         'sublead_ele_tt': h5_tt,
         'sublead_ele2_hct': h6_hct,
         'sublead_ele2_hut': h6_hut,
         'sublead_ele2_tt': h6_tt,
         'sublead_muon_hct': h7_hct,
         'sublead_muon_hut': h7_hut,
         'sublead_muon_tt': h7_tt,
         'sublead_muon2_hct': h8_hct,
         'sublead_muon2_hut': h8_hut,
         'sublead_muon2_tt': h8_tt,
         'lead_lep_hct': h9_hct,
         'lead_lep_hut': h9_hut,
         'lead_lep_tt': h9_tt,
         'lead_lep2_hct': h10_hct,
         'lead_lep2_hut': h10_hut,
         'lead_lep2_tt': h10_tt,
         'sublead_lep_hct': h11_hct,
         'sublead_lep_hut': h11_hut,
         'sublead_lep_tt': h11_tt,
         'sublead_lep2_hct': h12_hct,
         'sublead_lep2_hut': h12_hut,
         'sublead_lep2_tt': h12_tt,
}

hists['lead_ele_tt'].label = '$TT$'
hists['lead_ele_tt'].color = '#8AC926'

hists['lead_ele2_tt'].label = '$TT$'
hists['lead_ele2_tt'].color = '#659e0b'

hists['lead_muon_tt'].label = '$TT$'
hists['lead_muon_tt'].color = '#8AC926'

hists['lead_muon2_tt'].label = '$TT$'
hists['lead_muon2_tt'].color = '#659e0b'

hists['lead_lep_tt'].label = '$TT$'
hists['lead_lep_tt'].color = '#8AC926'

hists['lead_lep2_tt'].label = '$TT$'
hists['lead_lep2_tt'].color = '#659e0b'

hists['sublead_ele_tt'].label = '$TT$'
hists['sublead_ele_tt'].color = '#8AC926'

hists['sublead_ele2_tt'].label = '$TT$'
hists['sublead_ele2_tt'].color = '#659e0b'

hists['sublead_muon_tt'].label = '$TT$'
hists['sublead_muon_tt'].color = '#8AC926'

hists['sublead_muon2_tt'].label = '$TT$'
hists['sublead_muon2_tt'].color = '#659e0b'

hists['sublead_lep_tt'].label = '$TT$'
hists['sublead_lep_tt'].color = '#8AC926'

hists['sublead_lep2_tt'].label = '$TT$'
hists['sublead_lep2_tt'].color = '#659e0b'


keys1 = ['lead_ele_tt', 'lead_ele2_tt']
keys2 = ['lead_muon_tt', 'lead_muon2_tt']
keys3 = ['sublead_ele_tt', 'sublead_ele2_tt']
keys4 = ['sublead_muon_tt', 'sublead_muon2_tt']
keys5 = ['lead_lep_tt', 'lead_lep2_tt']
keys6 = ['sublead_lep_tt', 'sublead_lep2_tt']

ratio1 = (h2_hct+h2_hut).divide(h1_hct+h1_hut)
ratio2 = (h4_hct+h4_hut).divide(h3_hct+h3_hut)
ratio3 = (h6_hct+h6_hut).divide(h5_hct+h5_hut)
ratio4 = (h8_hct+h8_hut).divide(h7_hct+h7_hut)
ratio5 = (h10_hct+h10_hut).divide(h9_hct+h9_hut)
ratio6 = (h12_hct+h12_hut).divide(h11_hct+h11_hut)

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys1 ],
    hists['lead_ele_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys1 ],
    histtype="fill",
    stack=False,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys1],
    color=[ hists[x].color for x in keys1 ],
    ax=ax)

hep.histplot(
    [hists['lead_ele_hct'].counts],
    hists['lead_ele_hct'].edges,
    w2=(hists['lead_ele_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (dielectron)', np.sum(hists['lead_ele_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['lead_ele_hut'].counts],
    hists['lead_ele_hut'].edges,
    w2=(hists['lead_ele_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (dielectron)', np.sum(hists['lead_ele_hut'].counts))],
    color=['#808080'],
    ax=ax)


hep.histplot(
    [hists['lead_ele2_hct'].counts],
    hists['lead_ele2_hct'].edges,
    w2=(hists['lead_ele2_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (triggers)', np.sum(hists['lead_ele2_hct'].counts))],
    color=['#0000ff'],
    ax=ax)

hep.histplot(
    [hists['lead_ele2_hut'].counts],
    hists['lead_ele2_hut'].edges,
    w2=(hists['lead_ele2_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (triggers)', np.sum(hists['lead_ele2_hut'].counts))],
    color=['#9999ff'],
    ax=ax)

hep.histplot(
    [ratio1.counts],
    ratio1.edges,
    w2=ratio1.errors**2,
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.3)
rax.set_xlabel(r'$lead\ electron\ p_T\ (GeV)$')
rax.set_ylabel(r'Trig Efficiency')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys2 ],
    hists['lead_muon_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys2 ],
    histtype="fill",
    stack=False,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys2],
    color=[ hists[x].color for x in keys1 ],
    ax=ax)

hep.histplot(
    [hists['lead_muon_hct'].counts],
    hists['lead_muon_hct'].edges,
    w2=(hists['lead_muon_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (dimuon)', np.sum(hists['lead_muon_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['lead_muon_hut'].counts],
    hists['lead_muon_hut'].edges,
    w2=(hists['lead_muon_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (dimuon)', np.sum(hists['lead_muon_hut'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [hists['lead_muon2_hct'].counts],
    hists['lead_muon2_hct'].edges,
    w2=(hists['lead_muon2_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (triggers)', np.sum(hists['lead_muon2_hct'].counts))],
    color=['#0000ff'],
    ax=ax)

hep.histplot(
    [hists['lead_muon2_hut'].counts],
    hists['lead_muon2_hut'].edges,
    w2=(hists['lead_muon2_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (triggers)', np.sum(hists['lead_muon2_hut'].counts))],
    color=['#9999ff'],
    ax=ax)

hep.histplot(
    [ratio2.counts],
    ratio2.edges,
    w2=[ratio2.errors**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.3)
rax.set_xlabel(r'$lead\ muon\ p_T\ (GeV)$')
rax.set_ylabel(r'Trig Efficiency')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys3 ],
    hists['sublead_ele_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys3 ],
    histtype="fill",
    stack=False,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys3],
    color=[ hists[x].color for x in keys3 ],
    ax=ax)

hep.histplot(
    [hists['sublead_ele_hct'].counts],
    hists['sublead_ele_hct'].edges,
    w2=(hists['sublead_ele_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (dielectron)', np.sum(hists['sublead_ele_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['sublead_ele_hut'].counts],
    hists['sublead_ele_hut'].edges,
    w2=(hists['sublead_ele_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (dielectron)', np.sum(hists['sublead_ele_hut'].counts))],
    color=['#808080'],
    ax=ax)


hep.histplot(
    [hists['sublead_ele2_hct'].counts],
    hists['sublead_ele2_hct'].edges,
    w2=(hists['sublead_ele2_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (triggers)', np.sum(hists['sublead_ele2_hct'].counts))],
    color=['#0000ff'],
    ax=ax)

hep.histplot(
    [hists['sublead_ele2_hut'].counts],
    hists['sublead_ele2_hut'].edges,
    w2=(hists['sublead_ele2_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (triggers)', np.sum(hists['sublead_ele2_hut'].counts))],
    color=['#9999ff'],
    ax=ax)

hep.histplot(
    [ratio3.counts],
    ratio3.edges,
    w2=[ratio3.errors**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.3)
rax.set_xlabel(r'$sublead\ electron\ p_T\ (GeV)$')
rax.set_ylabel(r'Trig Efficiency')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys4 ],
    hists['sublead_muon_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys4 ],
    histtype="fill",
    stack=False,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys4],
    color=[ hists[x].color for x in keys3 ],
    ax=ax)

hep.histplot(
    [hists['sublead_muon_hct'].counts],
    hists['sublead_muon_hct'].edges,
    w2=(hists['sublead_muon_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (dimuon)', np.sum(hists['sublead_muon_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['sublead_muon_hut'].counts],
    hists['sublead_muon_hut'].edges,
    w2=(hists['sublead_muon_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (dimuon)', np.sum(hists['sublead_muon_hut'].counts))],
    color=['#808080'],
    ax=ax)


hep.histplot(
    [hists['sublead_muon2_hct'].counts],
    hists['sublead_muon2_hct'].edges,
    w2=(hists['sublead_muon2_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (triggers)', np.sum(hists['sublead_muon2_hct'].counts))],
    color=['#0000ff'],
    ax=ax)

hep.histplot(
    [hists['sublead_muon2_hut'].counts],
    hists['sublead_muon2_hut'].edges,
    w2=(hists['sublead_muon2_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (triggers)', np.sum(hists['sublead_muon2_hut'].counts))],
    color=['#9999ff'],
    ax=ax)


hep.histplot(
    [ratio4.counts],
    ratio4.edges,
    w2=[ratio4.errors**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.5,1.3)
rax.set_xlabel(r'$sublead\ muon\ p_T\ (GeV)$')
rax.set_ylabel(r'Trig Efficiency')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys5 ],
    hists['lead_lep_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys5 ],
    histtype="fill",
    stack=False,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys5],
    color=[ hists[x].color for x in keys5 ],
    ax=ax)

hep.histplot(
    [hists['lead_lep_hct'].counts],
    hists['lead_lep_hct'].edges,
    w2=(hists['lead_lep_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (dilep)', np.sum(hists['lead_lep_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['lead_lep_hut'].counts],
    hists['lead_lep_hut'].edges,
    w2=(hists['lead_lep_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (dilep)', np.sum(hists['lead_lep_hut'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [hists['lead_lep2_hct'].counts],
    hists['lead_lep2_hct'].edges,
    w2=(hists['lead_lep2_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (triggers)', np.sum(hists['lead_lep2_hct'].counts))],
    color=['#0000ff'],
    ax=ax)

hep.histplot(
    [hists['lead_lep2_hut'].counts],
    hists['lead_lep2_hut'].edges,
    w2=(hists['lead_lep2_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (triggers)', np.sum(hists['lead_lep2_hut'].counts))],
    color=['#9999ff'],
    ax=ax)

hep.histplot(
    [ratio5.counts],
    ratio5.edges,
    w2=[ratio5.errors**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.7,1.3)
rax.set_xlabel(r'$lead\ lep\ p_T\ (GeV)$')
rax.set_ylabel(r'Trig Efficiency')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
fig, (ax, rax) = plt.subplots(2,1,figsize=(10,10), gridspec_kw={"height_ratios": (3, 1), "hspace": 0.05}, sharex=True)
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys6 ],
    hists['sublead_lep_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys6 ],
    histtype="fill",
    stack=False,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys6],
    color=[ hists[x].color for x in keys5 ],
    ax=ax)

hep.histplot(
    [hists['sublead_lep_hct'].counts],
    hists['sublead_lep_hct'].edges,
    w2=(hists['sublead_lep_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (dilep)', np.sum(hists['sublead_lep_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['sublead_lep_hut'].counts],
    hists['sublead_lep_hut'].edges,
    w2=(hists['sublead_lep_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (dilep)', np.sum(hists['sublead_lep_hut'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [hists['sublead_lep2_hct'].counts],
    hists['sublead_lep2_hct'].edges,
    w2=(hists['sublead_lep2_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (triggers)', np.sum(hists['sublead_lep2_hct'].counts))],
    color=['#0000ff'],
    ax=ax)

hep.histplot(
    [hists['sublead_lep2_hut'].counts],
    hists['sublead_lep2_hut'].edges,
    w2=(hists['sublead_lep2_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (triggers)', np.sum(hists['sublead_lep2_hut'].counts))],
    color=['#9999ff'],
    ax=ax)

hep.histplot(
    [ratio6.counts],
    ratio6.edges,
    w2=[ratio6.errors**2],
    histtype="errorbar",
    color=['black'],
    ax=rax)

rax.set_ylim(0.7,1.3)
rax.set_xlabel(r'$sublead\ lep\ p_T\ (GeV)$')
rax.set_ylabel(r'Trig Efficiency')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp1 = scale_and_merge(output['j1'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp1 = tmp1.rebin('pt', pt_rebin2)
tmp1 = tmp1.rebin('eta', eta_rebin)

h1_hct = Hist1D.from_bincounts(
    tmp1.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp1.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h1_hut = Hist1D.from_bincounts(
    tmp1.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp1.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h1_hct = h1_hct*0.01
h1_hut = h1_hut*0.01

h1_tt = Hist1D.from_bincounts(
    tmp1.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp1.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('TT',)][1].T),
)

tmp2 = scale_and_merge(output['j2'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp2 = tmp2.rebin('pt', pt_rebin2)
tmp2 = tmp2.rebin('eta', eta_rebin)

h2_hct = Hist1D.from_bincounts(
    tmp2.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp2.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp2.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h2_hut = Hist1D.from_bincounts(
    tmp2.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp2.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp2.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h2_hct = h2_hct*0.01
h2_hut = h2_hut*0.01

h2_tt = Hist1D.from_bincounts(
    tmp2.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp2.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp2.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'all')[('TT',)][1].T),
)

tmp3 = scale_and_merge(output['j3'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp3 = tmp3.rebin('pt', pt_rebin2)
tmp3 = tmp3.rebin('eta', eta_rebin)

h3_hct = Hist1D.from_bincounts(
    tmp3.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hct',)].T,
    (tmp3.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp3.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h3_hut = Hist1D.from_bincounts(
    tmp3.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('hut',)].T,
    (tmp3.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp3.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h3_hct = h3_hct*0.01
h3_hut = h3_hut*0.01

h3_tt = Hist1D.from_bincounts(
    tmp3.sum('eta', 'phi', overflow = 'over').values(overflow = 'over')[('TT',)].T,
    (tmp3.axis('pt').edges(overflow = 'over')),
    errors = np.sqrt(tmp3.sum('eta', 'phi', overflow = 'over').values(sumw2=True, overflow = 'all')[('TT',)][1].T),
)

In [ ]:
hists = {'lead_jet_hct': h1_hct,
         'lead_jet_hut': h1_hut,
         'lead_jet_tt': h1_tt,
         'sublead_jet_hct': h2_hct,
         'sublead_jet_hut': h2_hut,
         'sublead_jet_tt': h2_tt,
         'subsublead_jet_hct': h3_hct,
         'subsublead_jet_hut': h3_hut,
         'subsublead_jet_tt': h3_tt,
}

hists['lead_jet_tt'].label = '$TTJets$'
hists['lead_jet_tt'].color = '#8AC926'

hists['sublead_jet_tt'].label = '$TTJets$'
hists['sublead_jet_tt'].color = '#8AC926'

hists['subsublead_jet_tt'].label = '$TTJets$'
hists['subsublead_jet_tt'].color = '#8AC926'


keys1 = ['lead_jet_tt']
keys2 = ['sublead_jet_tt']
keys3 = ['subsublead_jet_tt']

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys1 ],
    hists['lead_jet_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys1 ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys1],
    color=[ hists[x].color for x in keys1 ],
    ax=ax)

hep.histplot(
    [hists['lead_jet_hct'].counts],
    hists['lead_jet_hct'].edges,
    w2=(hists['lead_jet_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH', np.sum(hists['lead_jet_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['lead_jet_hut'].counts],
    hists['lead_jet_hut'].edges,
    w2=(hists['lead_jet_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH', np.sum(hists['lead_jet_hut'].counts))],
    color=['#0000ff'],
    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$lead\ jet\ p_T\ (GeV)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys2 ],
    hists['sublead_jet_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys2 ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys2],
    color=[ hists[x].color for x in keys2 ],
    ax=ax)

hep.histplot(
    [hists['sublead_jet_hct'].counts],
    hists['sublead_jet_hct'].edges,
    w2=(hists['sublead_jet_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH', np.sum(hists['sublead_jet_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['sublead_jet_hut'].counts],
    hists['sublead_jet_hut'].edges,
    w2=(hists['sublead_jet_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH', np.sum(hists['sublead_jet_hut'].counts))],
    color=['#0000ff'],
    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$sublead\ jet\ p_T\ (GeV)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys3 ],
    hists['subsublead_jet_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys3 ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys3],
    color=[ hists[x].color for x in keys3 ],
    ax=ax)

hep.histplot(
    [hists['subsublead_jet_hct'].counts],
    hists['subsublead_jet_hct'].edges,
    w2=(hists['subsublead_jet_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH', np.sum(hists['subsublead_jet_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['subsublead_jet_hut'].counts],
    hists['subsublead_jet_hut'].edges,
    w2=(hists['subsublead_jet_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH', np.sum(hists['subsublead_jet_hut'].counts))],
    color=['#0000ff'],
    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$subsublead\ jet\ p_T\ (GeV)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp1 = scale_and_merge(output['b1'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp1 = tmp1.rebin('pt', pt_rebin2)
tmp1 = tmp1.rebin('eta', eta_rebin)

h1_hct = Hist1D.from_bincounts(
    tmp1.sum('eta', 'phi', overflow = 'all').values(overflow = 'all')[('hct',)].T,
    (tmp1.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.sum('eta', 'phi', overflow = 'all').values(sumw2=True, overflow = 'all')[('hct',)][1].T),
)

h1_hut = Hist1D.from_bincounts(
    tmp1.sum('eta', 'phi', overflow = 'all').values(overflow = 'all')[('hut',)].T,
    (tmp1.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.sum('eta', 'phi', overflow = 'all').values(sumw2=True, overflow = 'all')[('hut',)][1].T),
)

h1_tt = Hist1D.from_bincounts(
    tmp1.sum('eta', 'phi', overflow = 'all').values(overflow = 'all')[('TT',)].T,
    (tmp1.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.sum('eta', 'phi', overflow = 'all').values(sumw2=True, overflow = 'all')[('TT',)][1].T),
)

h1_hct = h1_hct*0.01
h1_hut = h1_hut*0.01

tmp2 = scale_and_merge(output['b2'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp2 = tmp2.rebin('pt', pt_rebin2)
tmp2 = tmp2.rebin('eta', eta_rebin)

h2_hct = Hist1D.from_bincounts(
    tmp2.sum('eta', 'phi', overflow = 'all').values(overflow = 'all')[('hct',)].T,
    (tmp2.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('eta', 'phi', overflow = 'all').values(sumw2=True, overflow = 'all')[('hct',)][1].T),
)

h2_hut = Hist1D.from_bincounts(
    tmp2.sum('eta', 'phi', overflow = 'all').values(overflow = 'all')[('hut',)].T,
    (tmp2.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('eta', 'phi', overflow = 'all').values(sumw2=True, overflow = 'all')[('hut',)][1].T),
)

h2_tt = Hist1D.from_bincounts(
    tmp2.sum('eta', 'phi', overflow = 'all').values(overflow = 'all')[('TT',)].T,
    (tmp2.axis('pt').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('eta', 'phi', overflow = 'all').values(sumw2=True, overflow = 'all')[('TT',)][1].T),
)

h2_hct = h2_hct*0.01
h2_hut = h2_hut*0.01

In [ ]:
hists = {'lead_btag_hct': h1_hct,
         'lead_btag_hut': h1_hut,
         'lead_btag_tt': h1_tt,
         'sublead_btag_hct': h2_hct,
         'sublead_btag_hut': h2_hut,
         'sublead_btag_tt': h2_tt,
}

hists['lead_btag_tt'].label = '$TTJets$'
hists['lead_btag_tt'].color = '#8AC926'

hists['sublead_btag_tt'].label = '$TTJets$'
hists['sublead_btag_tt'].color = '#8AC926'


keys1 = ['lead_btag_tt']
keys2 = ['sublead_btag_tt']

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys1 ],
    hists['lead_btag_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys1 ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys1],
    color=[ hists[x].color for x in keys1 ],
    ax=ax)

hep.histplot(
    [hists['lead_btag_hct'].counts],
    hists['lead_btag_hct'].edges,
    w2=(hists['lead_btag_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (skim)', np.sum(hists['lead_btag_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['lead_btag_hut'].counts],
    hists['lead_btag_hut'].edges,
    w2=(hists['lead_btag_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (skim)', np.sum(hists['lead_btag_hut'].counts))],
    color=['#0000ff'],
    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$lead\ btag\ p_T\ (GeV)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys2 ],
    hists['sublead_btag_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys2 ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys2],
    color=[ hists[x].color for x in keys2 ],
    ax=ax)

hep.histplot(
    [hists['sublead_btag_hct'].counts],
    hists['sublead_btag_hct'].edges,
    w2=(hists['sublead_btag_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH (skim)', np.sum(hists['sublead_btag_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['sublead_btag_hut'].counts],
    hists['sublead_btag_hut'].edges,
    w2=(hists['sublead_btag_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH (skim)', np.sum(hists['sublead_btag_hut'].counts))],
    color=['#0000ff'],
    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$sublead\ btag\ p_T\ (GeV)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp1 = scale_and_merge(output['N_jet'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp1 = tmp1.rebin('multiplicity', N_bins_red)

h1_hct = Hist1D.from_bincounts(
    tmp1.values(overflow = 'all')[('hct',)].T,
    (tmp1.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'all')[('hct',)][1].T),
)

h1_hut = Hist1D.from_bincounts(
    tmp1.values(overflow = 'all')[('hut',)].T,
    (tmp1.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'all')[('hut',)][1].T),
)

h1_tt = Hist1D.from_bincounts(
    tmp1.values(overflow = 'all')[('TT',)].T,
    (tmp1.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'all')[('TT',)][1].T),
)

h1_hct = h1_hct*0.01
h1_hut = h1_hut*0.01

In [ ]:
hists = {'n_jets_hct': h1_hct,
         'n_jets_hut': h1_hut,
         'n_jets_tt': h1_tt,
}

hists['n_jets_tt'].label = '$TTJets$'
hists['n_jets_tt'].color = '#8AC926'

keys1 = ['n_jets_tt']

total_mc = get_total(hists, keys1)

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys1 ],
    hists['n_jets_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys1 ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys1],
    color=[ hists[x].color for x in keys1 ],
    ax=ax)

hep.histplot(
    [hists['n_jets_hct'].counts],
    hists['n_jets_hct'].edges,
    w2=(hists['n_jets_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH', np.sum(hists['n_jets_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['n_jets_hut'].counts],
    hists['n_jets_hut'].edges,
    w2=(hists['n_jets_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH', np.sum(hists['n_jets_hut'].counts))],
    color=['#0000ff'],
    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$N_{jets}$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

add_uncertainty(total_mc, ax)


ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp1 = scale_and_merge(output['N_b'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp1 = tmp1.rebin('multiplicity', N_bins_red)

h1_hct = Hist1D.from_bincounts(
    tmp1.values(overflow = 'all')[('hct',)].T,
    (tmp1.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'all')[('hct',)][1].T),
)

h1_hut = Hist1D.from_bincounts(
    tmp1.values(overflow = 'all')[('hut',)].T,
    (tmp1.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'all')[('hut',)][1].T),
)

h1_tt = Hist1D.from_bincounts(
    tmp1.values(overflow = 'all')[('TT',)].T,
    (tmp1.axis('multiplicity').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'all')[('TT',)][1].T),
)

h1_hct = h1_hct*0.01
h1_hut = h1_hut*0.01

In [ ]:
hists = {'n_b_hct': h1_hct,
         'n_b_hut': h1_hut,
         'n_b_tt': h1_tt,
}

hists['n_b_tt'].label = '$TTJets$'
hists['n_b_tt'].color = '#8AC926'

keys1 = ['n_b_tt']

total_mc = get_total(hists, keys1)

In [ ]:
fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

hep.histplot(
    [hists[x].counts for x in keys1 ],
    hists['n_b_tt'].edges,
    w2=[(hists[x].errors)**2 for x in keys1 ],
    histtype="fill",
    stack=True,
    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys1],
    color=[ hists[x].color for x in keys1 ],
    ax=ax)

hep.histplot(
    [hists['n_b_hct'].counts],
    hists['n_b_hct'].edges,
    w2=(hists['n_b_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH', np.sum(hists['n_b_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['n_b_hut'].counts],
    hists['n_b_hut'].edges,
    w2=(hists['n_b_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH', np.sum(hists['n_b_hut'].counts))],
    color=['#0000ff'],
    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$N_{b}$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

add_uncertainty(total_mc, ax)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp1 = output['dilep_mass1']
tmp1 = tmp1.rebin('mass', mass_bins)
tmp1 = tmp1.rebin('pt', pt_fine_bins)

h1 = Hist1D.from_bincounts(
    tmp1.sum('pt', 'dataset', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp1.axis('mass').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.sum('pt', 'dataset', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)



tmp2 = output['dilep_mass2']
tmp2 = tmp2.rebin('mass', mass_bins)
tmp2 = tmp2.rebin('pt', pt_fine_bins)

h2 = Hist1D.from_bincounts(
    tmp2.sum('pt', 'dataset', overflow = 'all').values(overflow = 'all')[()].T,
    (tmp2.axis('mass').edges(overflow = 'all')),
    errors = np.sqrt(tmp2.sum('pt', 'dataset', overflow = 'all').values(sumw2=True, overflow = 'all')[()][1].T),
)


In [ ]:
hists = {'mass_hct': h1,
         'mass_hut': h1,
}

fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

#hep.histplot(
#    [hists[x].counts for x in keys ],
#    hists['mass_DY_OS'].edges,
#    w2=[(hists[x].errors)**2 for x in keys ],
#    histtype="fill",
#    stack=True,
#    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
#    color=[ hists[x].color for x in keys ],
#    ax=ax)

hep.histplot(
    [hists['mass_hct'].counts],
    hists['mass_hct'].edges,
    w2=(hists['mass_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH', np.sum(hists['mass_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['mass_hut'].counts],
    hists['mass_hut'].edges,
    w2=(hists['mass_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH', np.sum(hists['mass_hut'].counts))],
    color=['#808080'],
    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$mass_{\ell\ell}\ (GeV)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
hists = {'mass_hct': h2,
         'mass_hut': h2,
}

fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

#hep.histplot(
#    [hists[x].counts for x in keys ],
#    hists['mass_DY_OS'].edges,
#    w2=[(hists[x].errors)**2 for x in keys ],
#    histtype="fill",
#    stack=True,
#    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
#    color=[ hists[x].color for x in keys ],
#    ax=ax)

hep.histplot(
    [hists['mass_hct'].counts],
    hists['mass_hct'].edges,
    w2=(hists['mass_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH', np.sum(hists['mass_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['mass_hut'].counts],
    hists['mass_hut'].edges,
    w2=(hists['mass_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH', np.sum(hists['mass_hut'].counts))],
    color=['#808080'],
    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$mass_{\ell\ell}\ (GeV)$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp1 = output['N_ele']
tmp1 = tmp1.rebin('multiplicity', N_bins_red)

tmp2 = output['N_jet']
tmp2 = tmp2.rebin('multiplicity', N_bins_red)

tmp3 = output['N_jet5']
tmp3 = tmp3.rebin('multiplicity', N_bins_red)


h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset').values(overflow = 'over')[()].T,
    (tmp1.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.sum('dataset').values(sumw2=True, overflow = 'over')[()][1].T),
)


h2 = Hist1D.from_bincounts(
    tmp2.sum('dataset').values(overflow = 'over')[()].T,
    (tmp2.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp2.sum('dataset').values(sumw2=True, overflow = 'over')[()][1].T),
)

h3 = Hist1D.from_bincounts(
    tmp3.sum('dataset').values(overflow = 'over')[()].T,
    (tmp3.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp3.sum('dataset').values(sumw2=True, overflow = 'over')[()][1].T),
)

In [ ]:
hists = {'n_lep_hct': h1,
         'n_lep_hut': h1,
         'n_loose_lep_hct': h2,
         'n_loose_lep_hut': h3,
}

fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

#hep.histplot(
#    [hists[x].counts for x in keys ],
#    hists['mass_DY_OS'].edges,
#    w2=[(hists[x].errors)**2 for x in keys ],
#    histtype="fill",
#    stack=True,
#    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
#    color=[ hists[x].color for x in keys ],
#    ax=ax)

hep.histplot(
    [hists['n_lep_hct'].counts],
    hists['n_lep_hct'].edges,
    w2=(hists['n_lep_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('tight', np.sum(hists['n_lep_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['n_lep_hut'].counts],
    hists['n_lep_hut'].edges,
    w2=(hists['n_lep_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('tight', np.sum(hists['n_lep_hut'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [hists['n_loose_lep_hct'].counts],
    hists['n_loose_lep_hct'].edges,
    w2=(hists['n_loose_lep_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('loose', np.sum(hists['n_loose_lep_hct'].counts))],
    color=['#0000ff'],
    ax=ax)

hep.histplot(
    [hists['n_loose_lep_hut'].counts],
    hists['n_loose_lep_hut'].edges,
    w2=(hists['n_loose_lep_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('loose not tight', np.sum(hists['n_loose_lep_hut'].counts))],
    color=['#9999ff'],
    ax=ax)


#
#

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$N_{\ell}$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp1 = output['N_ele2']
tmp1 = tmp1.rebin('multiplicity', N_bins_red)

tmp2 = output['N_jet2']
tmp2 = tmp2.rebin('multiplicity', N_bins_red)

tmp3 = output['N_jet6']
tmp3 = tmp3.rebin('multiplicity', N_bins_red)


h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset').values(overflow = 'over')[()].T,
    (tmp1.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.sum('dataset').values(sumw2=True, overflow = 'over')[()][1].T),
)


h2 = Hist1D.from_bincounts(
    tmp2.sum('dataset').values(overflow = 'over')[()].T,
    (tmp2.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp2.sum('dataset').values(sumw2=True, overflow = 'over')[()][1].T),
)

h3 = Hist1D.from_bincounts(
    tmp3.sum('dataset').values(overflow = 'over')[()].T,
    (tmp3.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp3.sum('dataset').values(sumw2=True, overflow = 'over')[()][1].T),
)

In [ ]:
hists = {'n_lep_hct': h1,
         'n_lep_hut': h1,
         'n_loose_lep_hct': h2,
         'n_loose_lep_hut': h3,
}

fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

#hep.histplot(
#    [hists[x].counts for x in keys ],
#    hists['mass_DY_OS'].edges,
#    w2=[(hists[x].errors)**2 for x in keys ],
#    histtype="fill",
#    stack=True,
#    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
#    color=[ hists[x].color for x in keys ],
#    ax=ax)

hep.histplot(
    [hists['n_lep_hct'].counts],
    hists['n_lep_hct'].edges,
    w2=(hists['n_lep_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('tight', np.sum(hists['n_lep_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['n_lep_hut'].counts],
    hists['n_lep_hut'].edges,
    w2=(hists['n_lep_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('tight', np.sum(hists['n_lep_hut'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [hists['n_loose_lep_hct'].counts],
    hists['n_loose_lep_hct'].edges,
    w2=(hists['n_loose_lep_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('loose', np.sum(hists['n_loose_lep_hct'].counts))],
    color=['#0000ff'],
    ax=ax)

hep.histplot(
    [hists['n_loose_lep_hut'].counts],
    hists['n_loose_lep_hut'].edges,
    w2=(hists['n_loose_lep_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('loose not tight', np.sum(hists['n_loose_lep_hut'].counts))],
    color=['#9999ff'],
    ax=ax)


#
#

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$N_{\ell}$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp1 = output['N_ele3']
tmp1 = tmp1.rebin('multiplicity', N_bins_red)

tmp2 = output['N_jet3']
tmp1 = tmp1.rebin('multiplicity', N_bins_red)


h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset').values(overflow = 'over')[()].T,
    (tmp1.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.sum('dataset').values(sumw2=True, overflow = 'over')[()][1].T),
)


h2 = Hist1D.from_bincounts(
    tmp2.sum('dataset').values(overflow = 'over')[()].T,
    (tmp2.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp2.sum('dataset').values(sumw2=True, overflow = 'over')[()][1].T),
)

In [ ]:
hists = {'n_lep_hct': h1,
         'n_lep_hut': h1,
         'n_loose_lep_hct': h2,
         'n_loose_lep_hut': h2,
}

fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

#hep.histplot(
#    [hists[x].counts for x in keys ],
#    hists['mass_DY_OS'].edges,
#    w2=[(hists[x].errors)**2 for x in keys ],
#    histtype="fill",
#    stack=True,
#    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
#    color=[ hists[x].color for x in keys ],
#    ax=ax)

hep.histplot(
    [hists['n_lep_hct'].counts],
    hists['n_lep_hct'].edges,
    w2=(hists['n_lep_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('tight', np.sum(hists['n_lep_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['n_lep_hut'].counts],
    hists['n_lep_hut'].edges,
    w2=(hists['n_lep_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('tight', np.sum(hists['n_lep_hut'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [hists['n_loose_lep_hct'].counts],
    hists['n_loose_lep_hct'].edges,
    w2=(hists['n_loose_lep_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('loose', np.sum(hists['n_loose_lep_hct'].counts))],
    color=['#0000ff'],
    ax=ax)

hep.histplot(
    [hists['n_loose_lep_hut'].counts],
    hists['n_loose_lep_hut'].edges,
    w2=(hists['n_loose_lep_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('loose', np.sum(hists['n_loose_lep_hut'].counts))],
    color=['#9999ff'],
    ax=ax)


#
#

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$N_{\ell}$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp1 = output['N_ele4']
tmp1 = tmp1.rebin('multiplicity', N_bins_red)

tmp2 = output['N_jet4']
tmp1 = tmp1.rebin('multiplicity', N_bins_red)


h1 = Hist1D.from_bincounts(
    tmp1.sum('dataset').values(overflow = 'over')[()].T,
    (tmp1.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.sum('dataset').values(sumw2=True, overflow = 'over')[()][1].T),
)


h2 = Hist1D.from_bincounts(
    tmp2.sum('dataset').values(overflow = 'over')[()].T,
    (tmp2.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp2.sum('dataset').values(sumw2=True, overflow = 'over')[()][1].T),
)

In [ ]:
hists = {'n_lep_hct': h1,
         'n_lep_hut': h1,
         'n_loose_lep_hct': h2,
         'n_loose_lep_hut': h2,
}

fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

#hep.histplot(
#    [hists[x].counts for x in keys ],
#    hists['mass_DY_OS'].edges,
#    w2=[(hists[x].errors)**2 for x in keys ],
#    histtype="fill",
#    stack=True,
#    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
#    color=[ hists[x].color for x in keys ],
#    ax=ax)

hep.histplot(
    [hists['n_lep_hct'].counts],
    hists['n_lep_hct'].edges,
    w2=(hists['n_lep_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('tight', np.sum(hists['n_lep_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['n_lep_hut'].counts],
    hists['n_lep_hut'].edges,
    w2=(hists['n_lep_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('tight', np.sum(hists['n_lep_hut'].counts))],
    color=['#808080'],
    ax=ax)

hep.histplot(
    [hists['n_loose_lep_hct'].counts],
    hists['n_loose_lep_hct'].edges,
    w2=(hists['n_loose_lep_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('loose', np.sum(hists['n_loose_lep_hct'].counts))],
    color=['#0000ff'],
    ax=ax)

hep.histplot(
    [hists['n_loose_lep_hut'].counts],
    hists['n_loose_lep_hut'].edges,
    w2=(hists['n_loose_lep_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('loose', np.sum(hists['n_loose_lep_hut'].counts))],
    color=['#9999ff'],
    ax=ax)


#
#

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$N_{\ell}$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp1 = scale_and_merge(output['N_jet'], meta, fileset, nano_mappings, lumi=lumi[year])
tmp1 = tmp1.rebin('multiplicity', N_bins_red)

h1_hct = Hist1D.from_bincounts(
    tmp1.values(overflow = 'over')[('hct',)].T,
    (tmp1.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'over')[('hct',)][1].T),
)

h1_hct = h1_hct*0.01

h1_hut = Hist1D.from_bincounts(
    tmp1.values(overflow = 'over')[('hut',)].T,
    (tmp1.axis('multiplicity').edges(overflow = 'over')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'over')[('hut',)][1].T),
)

h1_hut = h1_hut*0.01

In [ ]:
hists = {'n_jet_hct': h1_hct,
         'n_jet_hut': h1_hut,
}

fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

#hep.histplot(
#    [hists[x].counts for x in keys ],
#    hists['mass_DY_OS'].edges,
#    w2=[(hists[x].errors)**2 for x in keys ],
#    histtype="fill",
#    stack=True,
#    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
#    color=[ hists[x].color for x in keys ],
#    ax=ax)

hep.histplot(
    [hists['n_jet_hct'].counts],
    hists['n_jet_hct'].edges,
    w2=(hists['n_jet_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH', np.sum(hists['n_jet_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['n_jet_hut'].counts],
    hists['n_jet_hut'].edges,
    w2=(hists['n_jet_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH', np.sum(hists['n_jet_hut'].counts))],
    color=['#808080'],
    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$N_{jet}$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')

In [ ]:
tmp1 = scale_and_merge(output['charge'], meta, fileset, nano_mappings, lumi=lumi[year])

h1_hct = Hist1D.from_bincounts(
    tmp1.values(overflow = 'all')[('hct',)].T,
    (tmp1.axis('charge').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'all')[('hct',)][1].T),
)

h1_hct = h1_hct*0.01

h1_hut = Hist1D.from_bincounts(
    tmp1.values(overflow = 'all')[('hut',)].T,
    (tmp1.axis('charge').edges(overflow = 'all')),
    errors = np.sqrt(tmp1.values(sumw2=True, overflow = 'all')[('hut',)][1].T),
)

h1_hut = h1_hut*0.01

In [ ]:
hists = {'charge_hct': h1_hct,
         'charge_hut': h1_hut,
}

fig, (ax) = plt.subplots(1,1,figsize=(10,10))
hep.cms.label(
    data=False,
    year=year,
    lumi=lumi[year],
    loc=0,
    ax=ax,
)

#hep.histplot(
#    [hists[x].counts for x in keys ],
#    hists['mass_DY_OS'].edges,
#    w2=[(hists[x].errors)**2 for x in keys ],
#    histtype="fill",
#    stack=True,
#    label=['%s (%.0f)'%(hists[x].label, sum(hists[x].counts)) for x in keys],
#    color=[ hists[x].color for x in keys ],
#    ax=ax)

hep.histplot(
    [hists['charge_hct'].counts],
    hists['charge_hct'].edges,
    w2=(hists['charge_hct'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to cH', np.sum(hists['charge_hct'].counts))],
    color=['#525B76'],
    ax=ax)

hep.histplot(
    [hists['charge_hut'].counts],
    hists['charge_hut'].edges,
    w2=(hists['charge_hut'].errors)**2,
    histtype="step",
    stack=False,
    label=['%s (%.0f)'%('t to uH', np.sum(hists['charge_hut'].counts))],
    color=['#808080'],
    ax=ax)

#hep.histplot(
#    [ratio.counts],
#    ratio.edges,
#    w2=[hists['mass_OS_data'].counts/((hists['mass_OS_TT'].counts+hists['mass_OS_DY'].counts)**2)],
#    histtype="errorbar",
#    color=['black'],
#    ax=rax)

#rax.set_ylim(0.5,1.5)
ax.set_xlabel(r'$charge$')
#rax.set_ylabel(r'Data/MC')
ax.set_ylabel(r'Events')
#ax.set_yscale('log')
#ax.set_ylim(0.1,1e5)

ax.legend()

plt.show()

#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.png')
#fig.savefig('/home/users/ewallace/public_html/FCNC/zmass_data.pdf')